In [11]:
import sys
import os
import time
import json


import numpy as np
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.manifold import TSNE

from sklearn import tree

In [12]:
url = 'https://raw.githubusercontent.com/malogalbo/BasicTitanic/main/data/test.csv'
test_data = pd.read_csv(url, encoding='cp1252', index_col = 0, on_bad_lines = "skip")
url = 'https://raw.githubusercontent.com/malogalbo/BasicTitanic/main/data/train.csv'
train_data = pd.read_csv(url, encoding='cp1252', index_col = 0, on_bad_lines = "skip")

# Loading and Pre-Processing Data

In [13]:
def train_val_split(x, y, prop_vec, shuffle=True, seed=None):

    assert x.shape[0] == y.shape[0]
    prop_vec = prop_vec / np.sum(prop_vec) # normalize

    n = x.shape[0]
    n_train = int(np.ceil(n * prop_vec[0]))
    n_val = n - n_train

    assert np.amin([n_train, n_val]) >= 1   

    if shuffle:
        rng = np.random.default_rng(seed)
        pi = rng.permutation(n)
    else:
        pi = np.arange(0, n)

    pi_train = pi[0:n_train]
    pi_val = pi[n_train:n]

    train_x = x[pi_train]
    train_y = y[pi_train]

    val_x = x[pi_val]
    val_y = y[pi_val]  
    
    return train_x, train_y, val_x, val_y

One Hot Encode:
"Pclass" 3rd, "Embarked" 

From String to Bool:
"Sex" 5th

Normalize/Standardize
"Fare"
"Age"
"Parch"
"sibsp"


y-column:
"Survived" 2nd

In [14]:
#4,5,6 
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB
None


In [15]:
#Drop Name, Cabin, Ticket
names = ['Name', 'Cabin', 'Ticket']
train_data = train_data.drop(columns=names, axis=1)
test_data = test_data.drop(columns=names, axis=1)
print(train_data.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB
None


In [34]:
# Clean According To Above

# Extract y-column from training

# Then Split